In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def test(input1, input2):
    tfidf_vectorizer = TfidfVectorizer()
    lst = [input1, input2]
    tfidf_matrix = tfidf_vectorizer.fit_transform(lst)
    similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

    return similarity

print(test("tomato apple banana onion fish", "banana fish apple onion tomato"))

[[1.]]


In [43]:
import re

def not_include(lst, str):
    return all(ele not in str for ele in lst)

print (not_include(['soda', 'chicke'], "tomato apple banana onion fish"))

True


In [44]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

def lemmatization(ingredients):
    # nltk.download('wordnet')

    lemmatizer = WordNetLemmatizer()
    words_list = word_tokenize(ingredients)
    
    lemmatized_words_list = [lemmatizer.lemmatize(word) for word in words_list]
    lemmatized_words_str = ' '.join(lemmatized_words_list)

    return lemmatized_words_str

print(lemmatization("is are take took taken cat cats quiz quizzes divide divided dry dried"))
    

is are take took taken cat cat quiz quiz divide divided dry dried


In [45]:
import spacy

def lemmatization(ingredients):
    # spacy.cli.download("en_core_web_sm")
    # Load the English NLP model
    nlp = spacy.load("en_core_web_sm")

    # Process the text
    doc = nlp(ingredients)

    # Lemmatize each token
    lemmatized_words_list = [token.lemma_ for token in doc]
    lemmatized_words_str = ' '.join(lemmatized_words_list)
    
    return lemmatized_words_str

print(lemmatization("is are take took taken cat cats quiz quizzes divide divided dry dried"))

be be take take take cat cat quiz quiz divide divide dry dry


In [46]:
from PIL import Image

# Open an image file
image = Image.open('archive/images/miso-butter-roast-chicken-acorn-squash-panzanella.jpg')

# Display the image
image.show()

In [70]:
import pickle
import re

data = pickle.load(open("data.pkl", 'rb'))

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def recommend1(title, df):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['Cleaned_Ingredients'])
    similarity = cosine_similarity(tfidf_matrix)
    
    index = df[df['Title'] == title].index[0]
    recommended_list = sorted(list(enumerate(0.5 * similarity[index] + 0.5 * df['Rating'])), reverse=True, key=lambda x: x[1])
    recommended_recipes = []
    
    for i in recommended_list[0:5]:
        recommended_recipes.append(df.iloc[i[0]].Title)
    
    return similarity[index].shape

def recommend2(inputValue1, inputValue2, df):
    filtered_df = df.copy()

    def not_include(lst, str):
        return all(ele not in str for ele in lst)
    
    if inputValue1 != "":
        inputValue1_lst = re.split('[, ]+', inputValue1.lower())
        mask = filtered_df.apply(lambda row: not_include(inputValue1_lst, row['Cleaned_Ingredients']), axis=1)
        filtered_df = filtered_df[mask]
        
    tfidf_vectorizer = TfidfVectorizer()

    all_texts = filtered_df['Cleaned_Ingredients'].tolist() + [inputValue2]
    tfidf_matrix = tfidf_vectorizer.fit_transform(all_texts)

    input_vector = tfidf_matrix[-1]
    similarity = cosine_similarity(tfidf_matrix[:-1], input_vector)

    filtered_df['Similarity'] = similarity
    filtered_df['Recommendation_Metric'] = similarity.squeeze() + filtered_df['Rating']

    recommended_list = filtered_df.sort_values(by='Recommendation_Metric', ascending=False).head(5)
    recommended_recipes = []
    recommended_similarity = []
    
    for recipes in recommended_list['Title']:
        recommended_recipes.append(recipes)

    for sim in recommended_list['Recommendation_Metric']:
        recommended_similarity.append(sim)
        
    return recommended_similarity

print(recommend2("olive", "onion", data))
# print(recommend1("Miso-Butter Roast Chicken With Acorn Squash Panzanella", data))

ValueError: Data must be 1-dimensional, got ndarray of shape (8493, 8493) instead